In [21]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination


In [22]:
model = DiscreteBayesianNetwork(
    [
        ("Intelligence","Grade"),("StudyHours","Grade"),("Difficulty","Grade"),("Grade","Pass"),
    ]
)

In [23]:
cpd_int = TabularCPD("Intelligence",2,[[0.7],[0.3]],state_names={"Intelligence": ["High","Low"]})
cpd_std = TabularCPD("StudyHours",2,[[0.6],[0.4]],state_names={"StudyHours": ["Sufficient","Insufficient"]})
cpd_dif = TabularCPD("Difficulty",2,[[0.4],[0.6]],state_names={"Difficulty": ["Hard","Easy"]})

In [24]:
cpd_pass = TabularCPD(
    "Pass",2,[[0.95,0.80,0.50],[0.05,0.20,0.50]],evidence=["Grade"],evidence_card=[3],state_names={"Pass":["Yes","No"], "Grade":["A","B","C"]}
)
cpd_grade = TabularCPD(
    "Grade",3,
     [ [0.7, 0.5, 0.5, 0.3, 0.4, 0.2, 0.2, 0.1],  # A
        [0.2, 0.3, 0.3, 0.4, 0.4, 0.4, 0.3, 0.3],  # B
        [0.1, 0.2, 0.2, 0.3, 0.2, 0.4, 0.5, 0.6]   # C
    ],
     evidence=["Intelligence","StudyHours","Difficulty"],
     evidence_card=[2,2,2],
     state_names={
         "Grade" :["A","B","C"],
         "Intelligence": ["High","Low"],
         "StudyHours" : ["Sufficient","Insufficient"],
         "Difficulty" : ["Hard","Easy"]
     }
    
)

In [25]:
model.add_cpds(cpd_int,cpd_std,cpd_grade,cpd_std,cpd_pass,cpd_dif)
model.check_model()

inference = VariableElimination(model)

res = inference.query(variables=["Pass"],evidence={"StudyHours":"Sufficient","Difficulty": "Hard"})
print(res)
res = inference.query(variables=["Intelligence"],evidence={"Pass": "Yes"})
print(res)

+-----------+-------------+
| Pass      |   phi(Pass) |
+===========+=============+
| Pass(Yes) |      0.8525 |
+-----------+-------------+
| Pass(No)  |      0.1475 |
+-----------+-------------+
+--------------------+---------------------+
| Intelligence       |   phi(Intelligence) |
+====================+=====================+
| Intelligence(High) |              0.7285 |
+--------------------+---------------------+
| Intelligence(Low)  |              0.2715 |
+--------------------+---------------------+
